In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Kaggle". Running its setup...
Updating the package lists...
Installing nvidia-cuda-toolkit, this may take a few minutes...
Source files will be saved in "/tmp/tmpuzkpk_0j".


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!g++ -v

In [ ]:
%%writefile cpphw.hpp

#include <iostream>

void printHelloWorld()

{

std::cout << "HelloWorld\nHelloWorld";

}

In [ ]:
%%writefile cpphw.cpp

#include "cpphw.hpp"

int main()

{

printHelloWorld();

return 0;

}

In [ ]:
ls

In [ ]:
%%script bash
g++ cpphw.cpp -std=c++11 -o hw.out
./hw.out

# Day 1 (16 Feb, 2025)

In [ ]:
%%writefile vect_add_cpu.cpp
// Sequential Vector Addition in CPP

#include <iostream>
#include <vector>
#include <ctime>
using namespace std;

int main(){
    clock_t start = clock();
    int n = 200000000;
    vector<int> A(n), B(n), C(n);
    for (int i=0; i<n; i++){
        A[i] = i+1;
        B[i] = (i+1)*2;
    }

    for (int i=0;i<n; i++){
        C[i] = A[i] + B[i];

    }

    clock_t end = clock();

    double duration = static_cast<double>(end - start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "Time taken: " << duration * 1000 << " milliseconds" << std::endl; // Convert to milliseconds

    for (int i=0;i <20; i++){
        cout << C[i] << " ";
    }
    return 0;

}

In [ ]:
%%script bash
g++ vect_add_cpu.cpp -std=c++11 -o cpu.out
./cpu.out

In [ ]:
%%writefile vect_add_gpu.cu
// vector addition on gpu

#include <iostream>
#include <vector>
#include <ctime>
#include <cuda_runtime.h>

using namespace std;

__global__ void vector_addition(int *A, int *B, int *C, int n){
    // std::cout << "blockIdx.x = " << blockIdx.x << endl ;
    // std::cout << "blockDim.x = " << blockDim.x << endl ;
    // std::cout << "threadIdx.x = " << threadIdx.x << endl ;

    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i<n){
        C[i] = A[i] + B[i];
    }
    
};

int main(){
    int n = 200000000;
    vector<int> A(n), B(n), C(n);
    for (int i=0; i<n; i++){
        A[i] = i+1;
        B[i] = (i+1)*2;
    }

    clock_t start = clock();
    int *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, n*sizeof(int));
    cudaMalloc((void **)&d_B, n*sizeof(int));
    cudaMalloc((void **)&d_C, n*sizeof(int));

    cudaMemcpy(d_A, A.data(), n*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B.data(), n*sizeof(int), cudaMemcpyHostToDevice);

    int blockSize = 2;
    int gridSize = n+blockSize - 1;
    
    vector_addition <<<gridSize, blockSize>>>(d_A, d_B, d_C, n);
    cudaMemcpy(C.data(), d_C, n*sizeof(int), cudaMemcpyDeviceToHost);

    clock_t end = clock();

    double duration = static_cast<double>(end - start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "Time taken: " << duration * 1000 << " milliseconds" << std::endl; // Convert to milliseconds

    for (int i=0;i <20; i++){
        cout << C[i] << " ";
    }
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    
    return 0;  
}



In [ ]:
%%script bash
nvcc vect_add_gpu.cu -o kernel
./kernel

# Notes:
* Gird Dimension: gridDim.x, gridDim.y, gridDim.z is the number of blocks in each direction of Grid.
* Block Dimension: blockDim.x, blockDim.y, blockDim.z, is the number of threads in each direction of block.
* Block Index: blockIdx.x[y,z], index of the block within the grid
* Thread Index: threadIdx.x[y,z], index of thread within a block
  
threadId =

            blockDim.x\*blockIdx.x + blockDim.x\*threadIdx.y + threadIdx.x (simple 2D within a block) +
              
            gridDim.x\*blockDim.y\*blockIdx.y (blocks of threads in previous row) 


**Steps:**
1. Define the function to run on GPU.
2. Declare variables on CPU and allocate necessary memory on GPU
3. Copy the data from CPU to GPU on the allocated locations.
4. Call the kernel
5. Save the copy of the task result on the CPU.
6. free the memory on GPU


# Day 2 (17 Feb, 2025)

In [17]:
%%writefile vect_2D_add_gpu.cu
// vector addition on gpu

#include <iostream>
#include <vector>
#include <ctime>
#include <cuda_runtime.h>

using namespace std;

__global__ void vector_addition(int *A, int *B, int *C, int rows, int cols){
    // std::cout << "blockIdx.x = " << blockIdx.x << endl ;
    // std::cout << "blockDim.x = " << blockDim.x << endl ;
    // std::cout << "threadIdx.x = " << threadIdx.x << endl ;

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row < rows && col < cols){
        int index = row*cols + col;
       // std::cout << index << endl;
        C[index] = A[index] + B[index];
    }
    
};

void print_vector(vector<vector <int>> matrix){
  // vector<vector<int>> matrix(3);
    for (vector vec: matrix){
      for (int val : vec){
        cout << val << " ";
      }
      cout << endl;
    }
}

int main(){
    
    int rows = 4;
    int cols = 4;
    int n = rows*cols;
    vector<vector<int>> A = {{1,2,3,4}, {1,2,3,4}, {1,2,3,4}, {1,2,3,4}};
    vector<vector<int>> B(rows, vector<int>(cols, 1));
    vector<vector<int>> C(rows, vector<int>(cols, 0));


    clock_t start = clock();
    int *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, n*sizeof(int));
    cudaMalloc((void **)&d_B, n*sizeof(int));
    cudaMalloc((void **)&d_C, n*sizeof(int));

    cudaMemcpy(d_A, A.data(), n*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B.data(), n*sizeof(int), cudaMemcpyHostToDevice);

    dim3 blockSize(rows, cols); // Block dimension (threads per block in x and y) - adjust based on matrix size and GPU capabilities
    dim3 gridSize((cols + blockSize.x - 1) / blockSize.x, (rows + blockSize.y - 1) / blockSize.y);
    
    vector_addition <<<gridSize, blockSize>>>(d_A, d_B, d_C, rows, cols);
    cudaMemcpy(C.data(), d_C, n*sizeof(int), cudaMemcpyDeviceToHost);

    clock_t end = clock();

    double duration = static_cast<double>(end - start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "Time taken: " << duration * 1000 << " milliseconds" << std::endl; 
    // Convert to milliseconds

    cout << "CUDA Matrix Addition (2D):" << endl;
    // for (int i = 0; i < rows; ++i) {
     //    for (int j = 0; j < cols; ++j) {
     //        cout << C[i * cols + j] << " ";
    //     }
    //     cout << endl;
    // }
    print_vector(C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    
    return 0;  
}



Overwriting vect_2D_add_gpu.cu


In [18]:
%%script bash
nvcc vect_2D_add_gpu.cu -o vect_2d
./vect_2d

Time taken: 147.539 milliseconds
CUDA Matrix Addition (2D):


bash: line 2:  1934 Segmentation fault      (core dumped) ./vect_2d


CalledProcessError: Command 'b'nvcc vect_2D_add_gpu.cu -o vect_2d\n./vect_2d\n'' returned non-zero exit status 139.

Need to debug